In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/armut-data/armut_data.csv


In [2]:
df = pd.read_csv("/kaggle/input/armut-data/armut_data.csv")

In [3]:
df.head()

,UserId,ServiceId,CategoryId,CreateDate
0,25446,4,5,2017-08-06 16:11:00
1,22948,48,5,2017-08-06 16:12:00
2,10618,0,8,2017-08-06 16:13:00
3,7256,9,4,2017-08-06 16:14:00
4,25446,48,5,2017-08-06 16:16:00


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162523 entries, 0 to 162522
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   UserId      162523 non-null  int64 
 1   ServiceId   162523 non-null  int64 
 2   CategoryId  162523 non-null  int64 
 3   CreateDate  162523 non-null  object
dtypes: int64(3), object(1)
memory usage: 5.0+ MB


In [5]:
df['Hizmet'] = df['ServiceId'].astype(str) + '_' + df['CategoryId'].astype(str)

In [6]:
df["New_Date"] = pd.to_datetime(df["CreateDate"]).dt.to_period("M")
df['SepetID'] = df['UserId'].astype(str) + '_' + df["New_Date"].astype(str)

In [7]:
df2 = pd.crosstab(df['SepetID'], df['Hizmet'])

In [8]:
df2[df2 != 0] = 1

check

In [9]:
from mlxtend.frequent_patterns import apriori, association_rules

In [10]:
frequent_itemsets = apriori(df2,
                            min_support=0.01,
                            use_colnames=True)

frequent_itemsets.sort_values("support", ascending=False)[:10]

/opt/conda/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
8,0.238121,(18_4)
19,0.130286,(2_0)
5,0.120963,(15_1)
39,0.067762,(49_1)
28,0.066568,(38_4)
3,0.056627,(13_11)
12,0.047515,(22_0)
9,0.045563,(19_6)
15,0.042895,(25_0)
7,0.041533,(17_5)


In [11]:
rules = association_rules(frequent_itemsets,
                          metric="support",
                          min_threshold=0.01)

rules[rules["lift"] > 2].sort_values("lift", ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
10,(25_0),(22_0),0.042895,0.047515,0.011120,0.259247,5.456141,0.009082,1.285834,0.853324
11,(22_0),(25_0),0.047515,0.042895,0.011120,0.234043,5.456141,0.009082,1.249553,0.857462
19,(38_4),(9_4),0.066568,0.041393,0.010067,0.151234,3.653623,0.007312,1.129413,0.778096
18,(9_4),(38_4),0.041393,0.066568,0.010067,0.243216,3.653623,0.007312,1.233418,0.757661
4,(15_1),(33_4),0.120963,0.027310,0.011233,0.092861,3.400299,0.007929,1.072262,0.803047
5,(33_4),(15_1),0.027310,0.120963,0.011233,0.411311,3.400299,0.007929,1.493211,0.725728
12,(2_0),(22_0),0.130286,0.047515,0.016568,0.127169,2.676409,0.010378,1.091260,0.720197
13,(22_0),(2_0),0.047515,0.130286,0.016568,0.348700,2.676409,0.010378,1.335350,0.657611
14,(25_0),(2_0),0.042895,0.130286,0.013437,0.313257,2.404371,0.007849,1.266432,0.610268
15,(2_0),(25_0),0.130286,0.042895,0.013437,0.103136,2.404371,0.007849,1.067168,0.671590


In [12]:
latest_user_with_2_0 = df[df['Hizmet'] == '2_0'].sort_values(by='New_Date', ascending=False).iloc[0]['UserId']


def arl_recommender(rules_df, Services, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == Services:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

    return recommendation_list[0:rec_count]



recommendations = arl_recommender(rules, '2_0', rec_count=1)

print(f"Recommendation for {latest_user_with_2_0 } = {recommendations}")

Recommendation for 10591 = ['22_0']
